In [1]:
one_time_pad = [
    "48090", "79776", "45366", "46827", "11680",
    "68496", "47757", "10126", "36660", "25066",
    "07418", "79781", "48209", "28600", "65589",
    "04417", "18375", "89891", "68548", "65437",
    "96152", "81871", "38849", "23191", "35777",
    "59888", "98186", "01174", "19456", "73831",
    "74345", "88365", "39797", "08166", "97776",
    "96571", "53718", "56970", "37940", "60539",
    "91243", "74502", "87465", "41884", "44533",
    "72057", "94612", "35304", "29054", "33274",
]

In [2]:
conversion_chart = {
    "0": "CODE",
    "1": "A",
    "2": "E",
    "3": "I",
    "4": "N",
    "5": "O",
    "6": "T",
    "70": "B",
    "71": "C",
    "72": "D",
    "73": "F",
    "74": "G",
    "75": "H",
    "76": "J",
    "77": "K",
    "78": "L",
    "79": "M",
    "80": "P",
    "81": "Q",
    "82": "R",
    "83": "S",
    "84": "U",
    "85": "V",
    "86": "W",
    "87": "X",
    "88": "Y",
    "89": "Z",
    "90": "FIG",
    "91": ".",
    "92": ":",
    "93": "\"",
    "94": " ",
    "95": "+",
    "96": "-",
    "97": "=",
    "98": "REQ",
    "99": " ",
}

In [3]:
codebook = {
    "000": "ABORT",
    "019": "ACCEPT",
    "028": "ACCESS",
    "037": "ADDRESS",
    "253": "DECODE",
    "262": "DELAY",
    "271": "DIFFICULT",
    "280": "DOCUMENT",
    "046": "AFFIRMATIVE",
    "299": "ENCODE",
    "055": "AGENT",
    "307": "EVENING",
    "064": "AIRPLANE",
    "316": "EXECUTE",
    "073": "AIRPORT",
    "082": "ANSWER",
    "091": "AUTHORITY",
    "109": "BETWEEN",
    "118": "BORDER",
    "127": "BUILDING",
    "136": "CANCEL",
    "145": "CHANGE",
    "154": "CIVILIAN",
    "325": "FACTORY",
    "334": "FAILED",
    "343": "FERRY",
    "352": "FLIGHT",
    "361": "FREQUENCY",
    "370": "HARBOUR",
    "389": "HELICOPTER",
    "398": "HIGHWAY",
    "406": "IDENTITY",
    "163": "COMPROMISE",
    "415": "IMMEDIATE",
    "172": "COMPUTER",
    "181": "CONFIRM",
    "190": "CONTACT",
    "424": "IMPOSSIBLE",
    "505": "MILITARY",
    "514": "MONEY",
    "523": "MONTH",
    "532": "MORNING",
    "541": "MORSE",
    "550": "NEGATIVE",
    "569": "NIGHT",
    "758": "STREET",
    "767": "SUBWAY",
    "776": "SUCCESS",
    "785": "SUPPLY",
    "794": "SUPPORT",
    "802": "TELEPHONE",
    "811": "TODAY",
    "578": "OBSERVATION",
    "820": "TOMORROW",
    "587": "PASSPORT",
    "596": "PERSON",
    "839": "TRAIN",
    "848": "TRANSFER",
    "604": "PHOTOGRAPH",
    "857": "TRANSMIT",
    "613": "POSITIVE",
    "622": "POSSIBLE",
    "631": "POWER",
    "640": "PRIORITY",
    "659": "PROBLEM",
    "668": "QUESTION",
    "677": "RADIO",
    "433": "INFORMATION",
    "686": "RECEIVE",
    "866": "TRAVEL",
    "875": "TRUCK",
    "884": "UNABLE TO",
    "893": "URGENT",
    "901": "VERIFY",
    "910": "WEEK",
    "929": "WITHIN",
    "938": "YESTERDAY",
    "442": "INSTRUCTIONS",
    "695": "RENDEZVOUS",
    "947": "MOSCOW",
    "208": "COORDINATE",
    "451": "LOCATE",
    "217": "COUNTRY",
    "226": "COVERT",
    "235": "CURRENT",
    "244": "DANGER",
    "956": "BERLIN",
    "712": "RESERVATION",
    "965": "PARIS",
    "703": "REPEAT",
    "460": "LOCATION",
    "479": "MAIL",
    "488": "MEETING",
    "497": "MESSAGE",
    "721": "ROUTINE",
    "730": "SATELLITE",
    "749": "SHIP",
    "974": "LONDON",
    "983": "ISTANBUL",
    "992": "PRAGUE"
 }

In [4]:
def add_key_to_message(message: str, one_time_pad: str = one_time_pad) -> str:
    
    # Find the index of the key indicator
    key_indicator = one_time_pad.index(message[0])

    # One by one, add the key to each group of 5 digits
    plaintext = []
    for idx, group in enumerate(message[1:]):
        plaintext_group = ""

        # Find the one-time pad group, offset by the key indicator and group index
        opt_key = one_time_pad[key_indicator + idx + 1]

        # Add the digits, from left to right, ignoring any tens
        for i in range(5):
            plaintext_group += str((int(group[i]) + int(opt_key[i])) % 10)

        # Add the plaintext group to the plaintext list
        plaintext.append(plaintext_group)

    return plaintext


def convert_to_plaintext(
        message: list[str], 
        one_time_pad: list[str] = one_time_pad,
        conversion_chart: dict[str, str] = conversion_chart
) -> str:

    decoded_plaintext = ""

    # Add the offset key to the message
    offset_message = add_key_to_message(message, one_time_pad=one_time_pad)

    # Deal with each group one character at a time
    group_list = list("".join(offset_message))
    
    # We process one or two characters at a time until there"s nothing left in the group
    current_chars = ""

    while len(group_list) > 0:
        # Grab a character
        current_chars += group_list.pop(0)

        # If it's a match, add the plaintext to the message and reset the current characters
        if current_chars in conversion_chart.keys():
            translated = conversion_chart[current_chars]

            # If a figure is coming, we have to decode the whole thing until the next FIG tag
            if translated == "FIG":
                decoded_plaintext += " "

                # While the next three characters are the same, decode them as numbers
                while len(set(group_list[:3])) == 1:
                    # Get the next three characters
                    figure, group_list = group_list[:3], group_list[3:]
                    # Add the figure to the decoded plaintext
                    decoded_plaintext += figure[0]

            elif translated == "CODE":
                codegroup, group_list = "".join(group_list[:3]), group_list[3:]
                decoded_plaintext += codebook.get(codegroup)

            else:
                decoded_plaintext += conversion_chart[current_chars]

            # Reset the characters being decoded
            current_chars = ""
        
    return decoded_plaintext

In [5]:
message = [
    "68496", "32579", "24623", "65030", "96188", "42672", "00212", "01749", "61591"
]

In [6]:
convert_to_plaintext(message, one_time_pad, conversion_chart)

'MEETING AT 14 PM IN NY..'

In [7]:
message = ["00000", "02089", "19191", "90111", "22233", "39091"]
convert_to_plaintext(message, one_time_pad=["00000", "00000", "00000", "00000", "00000", "00000", "00000"])

'COORDINATE... 123 .'